In [1]:
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# 导入resnet50

In [2]:
resnet50 = keras.applications.ResNet50(include_top=False, pooling='avg')

94765736/94765736 [==============================] - 8s 0us/step


# 配置自己的网络(未使数据处理相同)

In [3]:
num_classes = 10

In [4]:
model = keras.models.Sequential()
model.add(resnet50)
model.add(keras.layers.Dense(num_classes, activation='softmax'))

In [5]:
model.layers

In [6]:
# 把除最后一层之外的参数, 全部冻结. 
model.layers[0].trainable = False

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 20,490
Non-trainable params: 23,587,712
_________________________________________________________________


In [9]:
train_dir = r'C:/Users/Administrator/Desktop/data/archive/training/training/'
valid_dir =r'C:/Users/Administrator/Desktop/data/archive/validation/validation/'

# 图片数据生成器
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1. / 255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

height = 224
width = 224
channels = 3
batch_size = 32
num_classes = 10

train_generator = train_datagen.flow_from_directory(train_dir,
                                 target_size = (height, width),
                                 batch_size = batch_size,
                                 shuffle = True,
                                 seed = 7,
                                 class_mode = 'categorical')

valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1. / 255
)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                 target_size = (height, width),
                                 batch_size = batch_size,
                                 shuffle = True,
                                 seed = 7,
                                 class_mode = 'categorical')
print(train_generator.samples)
print(valid_generator.samples)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098
272


使用迁移学习, 对10种猴子进行识别效果很差. 
原因是因为预处理方式不一样.

In [10]:
history = model.fit(train_generator,
                   steps_per_epoch=train_generator.samples // batch_size,
                   epochs=10,
                   validation_data=valid_generator,
                   validation_steps = valid_generator.samples // batch_size
                   )

Epoch 1/10
34/34 [==============================] - 114s 3s/step - loss: 2.3944 - acc: 0.0985 - val_loss: 2.2873 - val_acc: 0.1133
Epoch 2/10
34/34 [==============================] - 74s 2s/step - loss: 2.2630 - acc: 0.1398 - val_loss: 2.2327 - val_acc: 0.1758
Epoch 3/10
34/34 [==============================] - 74s 2s/step - loss: 2.2345 - acc: 0.1529 - val_loss: 2.2277 - val_acc: 0.1523
Epoch 4/10
34/34 [==============================] - 74s 2s/step - loss: 2.2132 - acc: 0.1567 - val_loss: 2.1830 - val_acc: 0.1641
Epoch 5/10
34/34 [==============================] - 74s 2s/step - loss: 2.1876 - acc: 0.1576 - val_loss: 2.1699 - val_acc: 0.1641
Epoch 6/10
34/34 [==============================] - 75s 2s/step - loss: 2.1796 - acc: 0.1632 - val_loss: 2.1477 - val_acc: 0.1797
Epoch 7/10
34/34 [==============================] - 73s 2s/step - loss: 2.1558 - acc: 0.1839 - val_loss: 2.1556 - val_acc: 0.1758
Epoch 8/10
34/34 [==============================] - 75s 2s/step - loss: 2.1544 - acc: 0.1

# 配置自己的网络(数据处理相同)

In [11]:
tf.__version__

'2.9.1'

preprocessing_function = keras.applications.resnet50.preprocess_input

In [13]:
train_dir = r'C:/Users/Administrator/Desktop/data/archive/training/training/'
valid_dir =r'C:/Users/Administrator/Desktop/data/archive/validation/validation/'


# 图片数据生成器
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    vertical_flip = True,
    fill_mode = 'nearest'
)

height = 224
width = 224
channels = 3
batch_size = 32
num_classes = 10

train_generator = train_datagen.flow_from_directory(train_dir,
                                 target_size = (height, width),
                                 batch_size = batch_size,
                                 shuffle = True,
                                 seed = 7,
                                 class_mode = 'categorical')

valid_datagen = keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = keras.applications.resnet50.preprocess_input
)
valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                 target_size = (height, width),
                                 batch_size = batch_size,
                                 shuffle = True,
                                 seed = 7,
                                 class_mode = 'categorical')
print(train_generator.samples)
print(valid_generator.samples)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.
1098
272


keras.applications.ResNet50.preprocess_input

In [14]:
# 修改了预处理方式之后, 再跑.
history = model.fit(train_generator,
                   steps_per_epoch=train_generator.samples // batch_size,
                   epochs=10,
                   validation_data=valid_generator,
                   validation_steps = valid_generator.samples // batch_size
                   )

Epoch 1/10
34/34 [==============================] - 71s 2s/step - loss: 5.9819 - acc: 0.3593 - val_loss: 0.6998 - val_acc: 0.7852
Epoch 2/10
34/34 [==============================] - 73s 2s/step - loss: 0.4825 - acc: 0.8377 - val_loss: 0.2505 - val_acc: 0.9297
Epoch 3/10
34/34 [==============================] - 73s 2s/step - loss: 0.2455 - acc: 0.9259 - val_loss: 0.2090 - val_acc: 0.9531
Epoch 4/10
34/34 [==============================] - 73s 2s/step - loss: 0.1932 - acc: 0.9296 - val_loss: 0.1792 - val_acc: 0.9492
Epoch 5/10
34/34 [==============================] - 77s 2s/step - loss: 0.1988 - acc: 0.9371 - val_loss: 0.1649 - val_acc: 0.9648
Epoch 6/10
34/34 [==============================] - 74s 2s/step - loss: 0.1800 - acc: 0.9353 - val_loss: 0.1354 - val_acc: 0.9727
Epoch 7/10
34/34 [==============================] - 74s 2s/step - loss: 0.1605 - acc: 0.9437 - val_loss: 0.1442 - val_acc: 0.9453
Epoch 8/10
34/34 [==============================] - 74s 2s/step - loss: 0.1314 - acc: 0.96

# 数据预测

In [16]:
for i in range(1):
    x, y = train_generator.next()
    print(type(x), type(y))
    print(x.shape, y.shape)
#     print(x)
    print(x.min())
    print(x.max())

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(32, 224, 224, 3) (32, 10)
-123.68
151.061


In [18]:
monkey = plt.imread('./n2060.jpg')

In [19]:
# 预测
monkey.shape

(501, 750, 3)

In [20]:
from  scipy import ndimage

In [ ]:
# 旋转
ndimage.rotate()
ndimage.shift()
ndimage.zoom()

In [21]:
# 224 = 501* x
# 224 = 750 * y
x = 224 / 501
y = 224 / 750

In [22]:
# 501, 750
# 224 
# zoom = (x, y)
monkey_zoomed = ndimage.zoom(monkey, (x, y, 1))

In [23]:
monkey_zoomed.min()

0

In [24]:
monkey = keras.applications.resnet50.preprocess_input(monkey_zoomed)

In [25]:
monkey.max()

151.061

In [26]:
monkey.min()

-123.68

In [27]:
monkey = monkey.reshape(1, 224, 224, 3)

主要是形状和尺寸不对.
先改变尺寸. 

再改变形状. reshape

In [28]:
model.predict(monkey).argmax(axis=1)

1/1 [==============================] - 1s 720ms/step


array([2], dtype=int64)

In [29]:
monkey_zoomed.shape

(224, 224, 3)

# 迁移学习之训练最后几层参数

In [30]:
resnet50 = keras.applications.ResNet50(include_top=False, pooling='avg', weights='imagenet')
resnet50.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, None, None,   0           ['input_2[0][0]']                
                                3)                                                                
                                                                                                  
 conv1_conv (Conv2D)            (None, None, None,   9472        ['conv1_pad[0][0]']              
                                64)                                                        

                                                                                                  
 conv2_block2_add (Add)         (None, None, None,   0           ['conv2_block1_out[0][0]',       
                                256)                              'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, None, None,   0           ['conv2_block2_add[0][0]']       
                                256)                                                              
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, None, None,   16448       ['conv2_block2_out[0][0]']       
                                64)                                                               
                                                                                                  
 conv2_blo

                                128)                                                              
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, None, None,   512        ['conv3_block2_2_conv[0][0]']    
 ization)                       128)                                                              
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, None, None,   0          ['conv3_block2_2_bn[0][0]']      
 n)                             128)                                                              
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, None, None,   66048       ['conv3_block2_2_relu[0][0]']    
                                512)                                                              
          

 conv4_block1_1_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_1_conv[0][0]']    
 ization)                       256)                                                              
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, None, None,   0          ['conv4_block1_1_bn[0][0]']      
 n)                             256)                                                              
                                                                                                  
 conv4_block1_2_conv (Conv2D)   (None, None, None,   590080      ['conv4_block1_1_relu[0][0]']    
                                256)                                                              
                                                                                                  
 conv4_block1_2_bn (BatchNormal  (None, None, None,   1024       ['conv4_block1_2_conv[0][0]']    
 ization) 

                                                                                                  
 conv4_block3_3_bn (BatchNormal  (None, None, None,   4096       ['conv4_block3_3_conv[0][0]']    
 ization)                       1024)                                                             
                                                                                                  
 conv4_block3_add (Add)         (None, None, None,   0           ['conv4_block2_out[0][0]',       
                                1024)                             'conv4_block3_3_bn[0][0]']      
                                                                                                  
 conv4_block3_out (Activation)  (None, None, None,   0           ['conv4_block3_add[0][0]']       
                                1024)                                                             
                                                                                                  
 conv4_blo

 ization)                       256)                                                              
                                                                                                  
 conv4_block6_2_relu (Activatio  (None, None, None,   0          ['conv4_block6_2_bn[0][0]']      
 n)                             256)                                                              
                                                                                                  
 conv4_block6_3_conv (Conv2D)   (None, None, None,   263168      ['conv4_block6_2_relu[0][0]']    
                                1024)                                                             
                                                                                                  
 conv4_block6_3_bn (BatchNormal  (None, None, None,   4096       ['conv4_block6_3_conv[0][0]']    
 ization)                       1024)                                                             
          

 conv5_block3_1_conv (Conv2D)   (None, None, None,   1049088     ['conv5_block2_out[0][0]']       
                                512)                                                              
                                                                                                  
 conv5_block3_1_bn (BatchNormal  (None, None, None,   2048       ['conv5_block3_1_conv[0][0]']    
 ization)                       512)                                                              
                                                                                                  
 conv5_block3_1_relu (Activatio  (None, None, None,   0          ['conv5_block3_1_bn[0][0]']      
 n)                             512)                                                              
                                                                                                  
 conv5_block3_2_conv (Conv2D)   (None, None, None,   2359808     ['conv5_block3_1_relu[0][0]']    
          

## 倒数五层自己训练

In [31]:
for layer in resnet50.layers[0:-5]:
    layer.trainable = False
    
resnet50_new = keras.models.Sequential([resnet50, keras.layers.Dense(10, activation='softmax')])
resnet50_new.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
resnet50_new.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_1 (Dense)             (None, 10)                20490     
                                                                 
Total params: 23,608,202
Trainable params: 1,075,210
Non-trainable params: 22,532,992
_________________________________________________________________


 ## 训练最后五层, 再跑.

In [32]:
history = resnet50_new.fit(train_generator,
                   steps_per_epoch=train_generator.samples // batch_size,
                   epochs=10,
                   validation_data=valid_generator,
                   validation_steps = valid_generator.samples // batch_size
                   )

Epoch 1/10
34/34 [==============================] - 81s 2s/step - loss: 0.6971 - acc: 0.7833 - val_loss: 0.0741 - val_acc: 0.9766
Epoch 2/10
34/34 [==============================] - 76s 2s/step - loss: 0.1738 - acc: 0.9475 - val_loss: 0.0565 - val_acc: 0.9844
Epoch 3/10
34/34 [==============================] - 75s 2s/step - loss: 0.1246 - acc: 0.9634 - val_loss: 0.0827 - val_acc: 0.9727
Epoch 4/10
34/34 [==============================] - 74s 2s/step - loss: 0.0933 - acc: 0.9728 - val_loss: 0.0502 - val_acc: 0.9844
Epoch 5/10
34/34 [==============================] - 74s 2s/step - loss: 0.1038 - acc: 0.9728 - val_loss: 0.0445 - val_acc: 0.9883
Epoch 6/10
34/34 [==============================] - 73s 2s/step - loss: 0.0797 - acc: 0.9700 - val_loss: 0.0734 - val_acc: 0.9766
Epoch 7/10
34/34 [==============================] - 74s 2s/step - loss: 0.0615 - acc: 0.9803 - val_loss: 0.0496 - val_acc: 0.9844
Epoch 8/10
34/34 [==============================] - 74s 2s/step - loss: 0.0634 - acc: 0.98

训练所有参数的时候也要加上include_top=False,然后把那个trainable=False注释掉就可以了吧

我后来训练出来训练集是90%，验证集是50% 